In [18]:
import itertools as it
import collections
from typing import List, Tuple
import doctest

In [43]:
import bs4
import urllib

def get_title(day):
    f = urllib.request.urlopen(f"https://adventofcode.com/2018/day/{day}")
    soup = bs4.BeautifulSoup(f.read())
    print(soup.h2.text.replace("-","").strip())    

## Day 1: Chronal Calibration

In [19]:
def day01_read(file_):
    return [int(line) for line in open(file_).readlines()]


def day01(xs: List[int]) -> int:
    return sum(xs)

In [20]:
inputs = day01_read("./2018/day01.txt")
day01(inputs)

497

In [21]:
def day01_mod(xs: List[int]) -> int:
    """
    >>> day01_mod([1, -1])
    0
    
    >>> day01_mod([+3, +3, +4, -2, -4])
    10
    
    >>> day01_mod([-6, +3, +8, +5, -6])
    5
    
    >>> day01_mod([+7, +7, -2, -7, -4])
    14
    """
    freq = 0
    seen = {freq}
    
    for x in it.cycle(xs):
        freq += x
        if freq in seen:
            break
        else:
            seen.add(freq)
    return freq

In [22]:
doctest.testmod()

TestResults(failed=0, attempted=4)

In [23]:
day01_mod(inputs)

558

## Day 2: Inventory Management System

In [12]:
def day02_read(file_: str) -> List[str]:
    return [line.strip() for line in open(file_).readlines()]


def day02(ss: List[str]) -> int:
    xs = [frozenset(collections.Counter(s).values()) for s in ss]
    twos = sum(1 for x in xs if 2 in x)
    threes = sum(1 for x in xs if 3 in x)
    return twos * threes

In [24]:
inputs = day02_read("./2018/day02.txt")
day02(inputs)

4940

In [14]:
len(inputs)

250

In [15]:
collections.Counter([len(s) for s in inputs])

Counter({26: 250})

So, all strings have the same length 26.

In [25]:
def day02_mod(ss: List[str]) -> str:

    def _similarity(pair):
        s, t = pair
        return sum(c1 == c2 for c1, c2 in zip(s, t))

    def _common_str(s, t):
        return "".join(c1 for c1, c2 in zip(s, t) if c1 == c2)

    size = len(ss)
    s, t = max([(s, t) for s, t in it.combinations(ss, 2)], 
              key=_similarity)
    return _common_str(s, t)

In [26]:
day02_mod(inputs)

'wrziyfdmlumeqvaatbiosngkc'

## Day 3: No Matter How You Slice It

In [33]:
def day03_read(file_):
    return [line.strip() for line in open("./2018/day03.txt").readlines()]
    
def day03_parse_line(line):
    """
    >>> day03_parse_line('#34 @ 23,699: 21x18')
    (34, 23, 699, 21, 18)
    """
    head, rest = line.split("@")
    id_ = int(head[1:])
    loc, size = rest.split(":")
    x, y = map(int, loc.split(","))
    dx, dy = map(int, size.split("x"))
    return id_, x, y, dx, dy

In [34]:
doctest.testmod()

TestResults(failed=0, attempted=5)

In [31]:
inputs = day03_read("./2018/day03.txt")
inputs[:5]

['#1 @ 871,327: 16x20',
 '#2 @ 676,717: 27x26',
 '#3 @ 245,818: 19x21',
 '#4 @ 89,306: 22x11',
 '#5 @ 451,712: 20x11']

In [19]:
def day03(lines):
    # いもす法
    d = collections.defaultdict(int)
    for line in lines:
        _, x, y, dx, dy = day03_parse(line)
        d[x, y] += 1
        d[x + dx, y     ] -= 1
        d[x     , y + dy] -= 1
        d[x + dy, y + dy] += 1
    
    ub_x = max(tup[0] for tup in d.keys()) + 1
    ub_y = max(tup[1] for tup in d.keys()) + 1
    
    res = 0
    for i in range(ub_x):
        for j in range(ub_y):
            d[i, j] += d[i, j - 1]

    xss = [[0] * (ub_x) for _ in range(ub_y)] 
    for i in range(ub_x):
        for j in range(ub_y):
            d[i, j] += d[i - 1, j]
            if d[i, j] >= 2:
                res += 1
            xss[j][i] = d[i, j]
            
#     for xs in xss:
#         print(xs)
    return res

In [20]:
xss = [
"#1 @ 1,3: 4x4",
"#2 @ 3,1: 4x4",
"#3 @ 5,5: 2x2",
]
day03(xss)

4

In [21]:
day03(inputs)

256452

In [ ]:
def day03_mod(lines):
    ...

## Day 4: Repose Record

In [36]:
def day04_read(file_):
    return sorted([line.strip() for line in open(file_).readlines()])

In [38]:
inputs = day04_read("./2018/day04.txt")
inputs[:10]

['[1518-03-09 23:46] Guard #727 begins shift',
 '[1518-03-10 00:05] falls asleep',
 '[1518-03-10 00:34] wakes up',
 '[1518-03-10 00:56] falls asleep',
 '[1518-03-10 00:58] wakes up',
 '[1518-03-10 23:57] Guard #691 begins shift',
 '[1518-03-11 00:10] falls asleep',
 '[1518-03-11 00:48] wakes up',
 '[1518-03-12 00:04] Guard #1987 begins shift',
 '[1518-03-12 00:21] falls asleep']

In [39]:
def _day04_parse(line):
    date = line[1:11]
    time = line[12:17]
    minute = int(time[-2:])
    msg = line[19:]
    return date, minute, msg

def _day04_guard_id(msg):
    return int(msg.split()[1][1:])

def day04(record):
    """    
    returns dictionary {guard_id: list of number of sleeps from 00:00 to 00:59}
    
    いもす法
    """
    x = collections.defaultdict(lambda: [0]*60)
    guard = -1
    for line in record:
        date, minute, msg = _day04_parse(line)
        if msg.startswith("Guard"):
            guard = _day04_guard_id(msg)
        elif msg.startswith("falls"):
            x[guard][minute] += 1
        elif msg.startswith("wakes"):
            x[guard][minute] -= 1
        else:
            raise ValueError("Failed to decode message")

    for guard in x:
        x[guard] = list(it.accumulate(x[guard]))

    return x

In [40]:
x = day04(inputs)
guard = max(x.keys(), key=lambda g: sum(x[g]))
print(guard)
print(x[guard])
minute = max(range(60), key=lambda i: x[guard][i])
guard * minute

1987
[1, 1, 1, 2, 3, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 8, 9, 9, 10, 10, 10, 11, 11, 13, 13, 13, 13, 13, 13, 15, 13, 13, 13, 13, 13, 12, 11, 12, 12, 13, 12, 12, 11, 11, 11, 10, 11, 10, 7, 6, 4, 4, 4, 2, 0]


67558

In [41]:
x = day04(inputs)
guard = max(x.keys(), key=lambda g: max(x[g]))
print(guard)
print(x[guard])
minute = max(range(60), key=lambda i: x[guard][i])
guard * minute

2633
[1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 6, 7, 6, 6, 6, 6, 6, 6, 6, 7, 7, 10, 12, 13, 14, 15, 17, 16, 15, 14, 12, 11, 11, 10, 10, 9, 7, 8, 8, 8, 9, 9, 7, 7, 7, 6, 6, 6, 7, 5, 5, 5, 3, 1, 1, 0]


78990

In [44]:
get_title(5)

Day 5: Alchemical Reduction


## Day 5: Alchemical Reduction